## Setup

In [ ]:
! rm -r biotldr/
! git clone https://github.com/mahta-r/biotldr.git

In [ ]:
! rm -r fairseq/
! git clone https://github.com/pytorch/fairseq

In [ ]:
! cd fairseq/ ; pip install --editable .

In [ ]:
! cd biotldr/ ; pip install -r requirements.txt

In [ ]:
! pip install -U git+https://github.com/pltrdy/pyrouge

In [ ]:
! git clone https://github.com/pltrdy/files2rouge.git
% cd files2rouge
! python setup_rouge.py
! python setup.py install
% cd /content

In [ ]:
! sudo apt-get install libxml-parser-perl

In [ ]:
! pip install transformers datasets

## Data Preprocesing - SciTLDR

In [ ]:
%%bash

cd biotldr/SciTLDR-Data
chmod +x make_datafiles.sh

export TASK=SciTLDR-A
./make_datafiles.sh

export TASK=SciTLDR-AIC
./make_datafiles.sh

## Data Preprocessing - BioTLDR

In [ ]:
%%bash

cd biotldr/BioTLDR-Data

python create_data_folders.py BioTLDR.csv

chmod +x make_datafiles.sh

export TASK=BioTLDR-A
./make_datafiles.sh 

export TASK=BioTLDR-I
./make_datafiles.sh 

export TASK=BioTLDR-C
./make_datafiles.sh 

export TASK=BioTLDR-AIC
./make_datafiles.sh

## Baselines

In [ ]:
%%bash

mkdir biotldr/outputs
cd biotldr/
python scripts/baselines.py

In [ ]:
%%bash 

cd biotldr/
python scripts/cal-rouge.py outputs/bart-large_a_baseline.hypo BioTLDR-Data/BioTLDR-A/test.jsonl --workers 1

In [ ]:
%%bash 

cd biotldr/
python scripts/cal-rouge.py outputs/bart-large_i_baseline.hypo BioTLDR-Data/BioTLDR-I/test.jsonl --workers 1

In [ ]:
%%bash 

cd biotldr/
python scripts/cal-rouge.py outputs/bart-large_c_baseline.hypo BioTLDR-Data/BioTLDR-C/test.jsonl --workers 1

In [ ]:
%%bash 

cd biotldr/
python scripts/cal-rouge.py outputs/bart-large_aic_baseline.hypo BioTLDR-Data/BioTLDR-AIC/test.jsonl --workers 1

In [ ]:
%%bash 

cd biotldr/
python scripts/cal-rouge.py outputs/bart-large-xsum_a_baseline.hypo BioTLDR-Data/BioTLDR-A/test.jsonl --workers 1

In [ ]:
%%bash 

cd biotldr/
python scripts/cal-rouge.py outputs/bart-large-xsum_i_baseline.hypo BioTLDR-Data/BioTLDR-I/test.jsonl --workers 1

In [ ]:
%%bash 

cd biotldr/
python scripts/cal-rouge.py outputs/bart-large-xsum_c_baseline.hypo BioTLDR-Data/BioTLDR-C/test.jsonl --workers 1

In [ ]:
%%bash 

cd biotldr/
python scripts/cal-rouge.py outputs/bart-large-xsum_aic_baseline.hypo BioTLDR-Data/BioTLDR-AIC/test.jsonl --workers 1

## Experiments - Domain Transfer

In [ ]:
! mkdir biotldr/models 

### BART-large ▶ fine-tuned on SciTLDR (abstract only):

In [ ]:
! gsutil cp gs://skiff-models/scitldr/bart.tldr-ao.pt biotldr/models/bart.tldr-ao.pt

Copying gs://skiff-models/scitldr/bart.tldr-ao.pt...
- [1 files][  3.8 GiB/  3.8 GiB]   50.1 MiB/s                                   
Operation completed over 1 objects/3.8 GiB.                                      


In [ ]:
%%bash

cd biotldr/
cp -r SciTLDR-Data/SciTLDR-A-bin/ BioTLDR-Data/BioTLDR-A-bin/
python scripts/generate.py models/ BioTLDR-Data/BioTLDR-A ./outputs/ --checkpoint_file bart.tldr-ao.pt --beam 6 --lenpen 1.0 --test_fname bart_ao_domain_transfer.hypo

loading archive file models/
loading archive file BioTLDR-Data/BioTLDR-A-bin
| [source] dictionary: 50264 types
| [target] dictionary: 50264 types
Time to run script: 59.78709292411804 sec


152it [00:33,  4.56it/s]


In [ ]:
%%bash 

cd biotldr/
python scripts/cal-rouge.py outputs/bart_ao_domain_transfer.hypo BioTLDR-Data/BioTLDR-A/test.jsonl --workers 1

Preparing documents... 0 line(s) ignored
Running ROUGE...
---------------------------------------------
1 ROUGE-1 Average_R: 0.16667 (95%-conf.int. 0.16667 - 0.16667)
1 ROUGE-1 Average_P: 0.41176 (95%-conf.int. 0.41176 - 0.41176)
1 ROUGE-1 Average_F: 0.23729 (95%-conf.int. 0.23729 - 0.23729)
---------------------------------------------
1 ROUGE-2 Average_R: 0.02439 (95%-conf.int. 0.02439 - 0.02439)
1 ROUGE-2 Average_P: 0.06250 (95%-conf.int. 0.06250 - 0.06250)
1 ROUGE-2 Average_F: 0.03509 (95%-conf.int. 0.03509 - 0.03509)
---------------------------------------------
1 ROUGE-L Average_R: 0.14286 (95%-conf.int. 0.14286 - 0.14286)
1 ROUGE-L Average_P: 0.35294 (95%-conf.int. 0.35294 - 0.35294)
1 ROUGE-L Average_F: 0.20339 (95%-conf.int. 0.20339 - 0.20339)

Elapsed time: 0.132 seconds
Preparing documents... 0 line(s) ignored
Running ROUGE...
---------------------------------------------
1 ROUGE-1 Average_R: 0.21053 (95%-conf.int. 0.21053 - 0.21053)
1 ROUGE-1 Average_P: 0.38095 (95%-conf.in

100%|██████████| 153/153 [00:14<00:00, 10.67it/s]


### BART-large ▶ fine-tuned on SciTLDR (abstract, introduction, conclusion):

In [ ]:
! gsutil cp gs://skiff-models/scitldr/bart.tldr-aic.pt biotldr/models/bart.tldr-aic.pt

Copying gs://skiff-models/scitldr/bart.tldr-aic.pt...
| [1 files][  3.8 GiB/  3.8 GiB]   43.4 MiB/s                                   
Operation completed over 1 objects/3.8 GiB.                                      


In [ ]:
%%bash

cd biotldr/
cp -r SciTLDR-Data/SciTLDR-AIC-bin/ BioTLDR-Data/BioTLDR-AIC-bin/
python scripts/generate.py models/ BioTLDR-Data/BioTLDR-AIC ./outputs/ --checkpoint_file bart.tldr-aic.pt --beam 2 --lenpen 0.2 --test_fname bart_aic_domain_transfer.hypo

loading archive file models/
loading archive file BioTLDR-Data/BioTLDR-AIC-bin
| [source] dictionary: 50264 types
| [target] dictionary: 50264 types
Time to run script: 46.255047082901 sec


151it [00:22,  6.61it/s]


In [ ]:
%%bash 

cd biotldr/
python scripts/cal-rouge.py outputs/bart_aic_domain_transfer.hypo BioTLDR-Data/BioTLDR-AIC/test.jsonl --workers 1

Preparing documents... 0 line(s) ignored
Running ROUGE...
---------------------------------------------
1 ROUGE-1 Average_R: 0.16667 (95%-conf.int. 0.16667 - 0.16667)
1 ROUGE-1 Average_P: 0.33333 (95%-conf.int. 0.33333 - 0.33333)
1 ROUGE-1 Average_F: 0.22222 (95%-conf.int. 0.22222 - 0.22222)
---------------------------------------------
1 ROUGE-2 Average_R: 0.02439 (95%-conf.int. 0.02439 - 0.02439)
1 ROUGE-2 Average_P: 0.05000 (95%-conf.int. 0.05000 - 0.05000)
1 ROUGE-2 Average_F: 0.03279 (95%-conf.int. 0.03279 - 0.03279)
---------------------------------------------
1 ROUGE-L Average_R: 0.09524 (95%-conf.int. 0.09524 - 0.09524)
1 ROUGE-L Average_P: 0.19048 (95%-conf.int. 0.19048 - 0.19048)
1 ROUGE-L Average_F: 0.12699 (95%-conf.int. 0.12699 - 0.12699)

Elapsed time: 0.094 seconds
Preparing documents... 0 line(s) ignored
Running ROUGE...
---------------------------------------------
1 ROUGE-1 Average_R: 0.21053 (95%-conf.int. 0.21053 - 0.21053)
1 ROUGE-1 Average_P: 0.34783 (95%-conf.in

100%|██████████| 152/152 [00:14<00:00, 10.72it/s]


### BART-large ▶ fine-tuned on XSum ▶ fine-tuned on SciTLDR (abstract only):


In [ ]:
! gsutil cp gs://skiff-models/scitldr/bart-xsum.tldr-ao.pt biotldr/models/bart-xsum.tldr-ao.pt

Copying gs://skiff-models/scitldr/bart-xsum.tldr-ao.pt...
- [1 files][  3.8 GiB/  3.8 GiB]   66.3 MiB/s                                   
Operation completed over 1 objects/3.8 GiB.                                      


In [ ]:
%%bash

cd biotldr/
cp -r SciTLDR-Data/SciTLDR-A-bin/ BioTLDR-Data/BioTLDR-A-bin/
python scripts/generate.py models/ BioTLDR-Data/BioTLDR-A ./outputs/ --checkpoint_file bart-xsum.tldr-ao.pt --beam 2 --lenpen 0.8 --test_fname bart_xsum_ao_domain_transfer.hypo

loading archive file models/
loading archive file BioTLDR-Data/BioTLDR-A-bin
| [source] dictionary: 50264 types
| [target] dictionary: 50264 types
Time to run script: 39.34453225135803 sec


152it [00:15, 10.00it/s]


In [ ]:
%%bash 

cd biotldr/
python scripts/cal-rouge.py outputs/bart_xsum_ao_domain_transfer.hypo BioTLDR-Data/BioTLDR-A/test.jsonl --workers 1

Preparing documents... 0 line(s) ignored
Running ROUGE...
---------------------------------------------
1 ROUGE-1 Average_R: 0.19048 (95%-conf.int. 0.19048 - 0.19048)
1 ROUGE-1 Average_P: 0.33333 (95%-conf.int. 0.33333 - 0.33333)
1 ROUGE-1 Average_F: 0.24243 (95%-conf.int. 0.24243 - 0.24243)
---------------------------------------------
1 ROUGE-2 Average_R: 0.02439 (95%-conf.int. 0.02439 - 0.02439)
1 ROUGE-2 Average_P: 0.04348 (95%-conf.int. 0.04348 - 0.04348)
1 ROUGE-2 Average_F: 0.03125 (95%-conf.int. 0.03125 - 0.03125)
---------------------------------------------
1 ROUGE-L Average_R: 0.09524 (95%-conf.int. 0.09524 - 0.09524)
1 ROUGE-L Average_P: 0.16667 (95%-conf.int. 0.16667 - 0.16667)
1 ROUGE-L Average_F: 0.12121 (95%-conf.int. 0.12121 - 0.12121)

Elapsed time: 0.095 seconds
Preparing documents... 0 line(s) ignored
Running ROUGE...
---------------------------------------------
1 ROUGE-1 Average_R: 0.21053 (95%-conf.int. 0.21053 - 0.21053)
1 ROUGE-1 Average_P: 0.34783 (95%-conf.in

100%|██████████| 153/153 [00:14<00:00, 10.75it/s]


### BART-large ▶ fine-tuned on XSum ▶ fine-tuned on SciTLDR (abstract, introduction, conclusion):

In [ ]:
! gsutil cp gs://skiff-models/scitldr/bart-xsum.tldr-aic.pt biotldr/models/bart-xsum.tldr-aic.pt

Copying gs://skiff-models/scitldr/bart-xsum.tldr-aic.pt...
\ [1 files][  3.8 GiB/  3.8 GiB]   91.4 MiB/s                                   
Operation completed over 1 objects/3.8 GiB.                                      


In [ ]:
%%bash

cd biotldr/
cp -r SciTLDR-Data/SciTLDR-AIC-bin/ BioTLDR-Data/BioTLDR-AIC-bin/
python scripts/generate.py models/ BioTLDR-Data/BioTLDR-AIC ./outputs/ --checkpoint_file bart-xsum.tldr-aic.pt --beam 5 --lenpen 0.8 --test_fname bart_xsum_aic_domain_transfer.hypo

loading archive file models/
loading archive file BioTLDR-Data/BioTLDR-AIC-bin
| [source] dictionary: 50264 types
| [target] dictionary: 50264 types
Time to run script: 68.50383019447327 sec


151it [00:40,  3.69it/s]


In [ ]:
%%bash 

cd biotldr/
python scripts/cal-rouge.py outputs/bart_xsum_aic_domain_transfer.hypo BioTLDR-Data/BioTLDR-AIC/test.jsonl --workers 1

Preparing documents... 0 line(s) ignored
Running ROUGE...
---------------------------------------------
1 ROUGE-1 Average_R: 0.16667 (95%-conf.int. 0.16667 - 0.16667)
1 ROUGE-1 Average_P: 0.41176 (95%-conf.int. 0.41176 - 0.41176)
1 ROUGE-1 Average_F: 0.23729 (95%-conf.int. 0.23729 - 0.23729)
---------------------------------------------
1 ROUGE-2 Average_R: 0.02439 (95%-conf.int. 0.02439 - 0.02439)
1 ROUGE-2 Average_P: 0.06250 (95%-conf.int. 0.06250 - 0.06250)
1 ROUGE-2 Average_F: 0.03509 (95%-conf.int. 0.03509 - 0.03509)
---------------------------------------------
1 ROUGE-L Average_R: 0.14286 (95%-conf.int. 0.14286 - 0.14286)
1 ROUGE-L Average_P: 0.35294 (95%-conf.int. 0.35294 - 0.35294)
1 ROUGE-L Average_F: 0.20339 (95%-conf.int. 0.20339 - 0.20339)

Elapsed time: 0.085 seconds
Preparing documents... 0 line(s) ignored
Running ROUGE...
---------------------------------------------
1 ROUGE-1 Average_R: 0.13158 (95%-conf.int. 0.13158 - 0.13158)
1 ROUGE-1 Average_P: 0.27778 (95%-conf.in

100%|██████████| 152/152 [00:13<00:00, 11.45it/s]


### BART-large ▶ fine-tuned on SciTLDR using CATTS framework (abstract only):

In [ ]:
! gsutil cp gs://skiff-models/scitldr/catts.tldr-ao.pt biotldr/models/catts.tldr-ao.pt

Copying gs://skiff-models/scitldr/catts.tldr-ao.pt...
| [1 files][  3.8 GiB/  3.8 GiB]   47.0 MiB/s                                   
Operation completed over 1 objects/3.8 GiB.                                      


In [ ]:
%%bash

cd biotldr/
cp -r SciTLDR-Data/SciTLDR-A/ctrl-bin/ BioTLDR-Data/BioTLDR-A/ctrl-bin/
python scripts/generate.py models/ BioTLDR-Data/BioTLDR-A/ctrl ./outputs/ --checkpoint_file catts.tldr-ao.pt --beam 2 --lenpen 0.4 --test_fname catts_ao_domain_transfer.hypo

loading archive file models/
loading archive file BioTLDR-Data/BioTLDR-A/ctrl-bin
| [source] dictionary: 50264 types
| [target] dictionary: 50264 types
Time to run script: 41.834041118621826 sec


1042301B [00:01, 1017566.28B/s]
456318B [00:00, 472602.10B/s]
152it [00:15, 10.09it/s]


In [61]:
%%bash 

cd biotldr/
python scripts/cal-rouge.py outputs/catts_ao_domain_transfer.hypo BioTLDR-Data/BioTLDR-A/test.jsonl --workers 1

Preparing documents... 0 line(s) ignored
Running ROUGE...
---------------------------------------------
1 ROUGE-1 Average_R: 0.19048 (95%-conf.int. 0.19048 - 0.19048)
1 ROUGE-1 Average_P: 0.33333 (95%-conf.int. 0.33333 - 0.33333)
1 ROUGE-1 Average_F: 0.24243 (95%-conf.int. 0.24243 - 0.24243)
---------------------------------------------
1 ROUGE-2 Average_R: 0.02439 (95%-conf.int. 0.02439 - 0.02439)
1 ROUGE-2 Average_P: 0.04348 (95%-conf.int. 0.04348 - 0.04348)
1 ROUGE-2 Average_F: 0.03125 (95%-conf.int. 0.03125 - 0.03125)
---------------------------------------------
1 ROUGE-L Average_R: 0.09524 (95%-conf.int. 0.09524 - 0.09524)
1 ROUGE-L Average_P: 0.16667 (95%-conf.int. 0.16667 - 0.16667)
1 ROUGE-L Average_F: 0.12121 (95%-conf.int. 0.12121 - 0.12121)

Elapsed time: 0.101 seconds
Preparing documents... 0 line(s) ignored
Running ROUGE...
---------------------------------------------
1 ROUGE-1 Average_R: 0.21053 (95%-conf.int. 0.21053 - 0.21053)
1 ROUGE-1 Average_P: 0.34783 (95%-conf.in

100%|██████████| 153/153 [00:14<00:00, 10.76it/s]


### BART-large ▶ fine-tuned on SciTLDR using CATTS framework (abstract, introduction, conclusion):

In [ ]:
! gsutil cp gs://skiff-models/scitldr/catts.tldr-aic.pt biotldr/models/catts.tldr-aic.pt

Copying gs://skiff-models/scitldr/catts.tldr-aic.pt...
\ [1 files][  3.8 GiB/  3.8 GiB]   23.2 MiB/s                                   
Operation completed over 1 objects/3.8 GiB.                                      


In [ ]:
%%bash

cd biotldr/
cp -r SciTLDR-Data/SciTLDR-AIC/ctrl-bin/ BioTLDR-Data/BioTLDR-AIC/ctrl-bin/
python scripts/generate.py models/ BioTLDR-Data/BioTLDR-AIC/ctrl ./outputs/ --checkpoint_file catts.tldr-aic.pt --beam 4 --lenpen 0.4 --test_fname catts_aic_domain_transfer.hypo

loading archive file models/
loading archive file BioTLDR-Data/BioTLDR-AIC/ctrl-bin
| [source] dictionary: 50264 types
| [target] dictionary: 50264 types
Time to run script: 58.41555333137512 sec


151it [00:32,  4.62it/s]


In [62]:
%%bash 

cd biotldr/
python scripts/cal-rouge.py outputs/catts_aic_domain_transfer.hypo BioTLDR-Data/BioTLDR-AIC/test.jsonl --workers 1

Preparing documents... 0 line(s) ignored
Running ROUGE...
---------------------------------------------
1 ROUGE-1 Average_R: 0.19048 (95%-conf.int. 0.19048 - 0.19048)
1 ROUGE-1 Average_P: 0.33333 (95%-conf.int. 0.33333 - 0.33333)
1 ROUGE-1 Average_F: 0.24243 (95%-conf.int. 0.24243 - 0.24243)
---------------------------------------------
1 ROUGE-2 Average_R: 0.02439 (95%-conf.int. 0.02439 - 0.02439)
1 ROUGE-2 Average_P: 0.04348 (95%-conf.int. 0.04348 - 0.04348)
1 ROUGE-2 Average_F: 0.03125 (95%-conf.int. 0.03125 - 0.03125)
---------------------------------------------
1 ROUGE-L Average_R: 0.09524 (95%-conf.int. 0.09524 - 0.09524)
1 ROUGE-L Average_P: 0.16667 (95%-conf.int. 0.16667 - 0.16667)
1 ROUGE-L Average_F: 0.12121 (95%-conf.int. 0.12121 - 0.12121)

Elapsed time: 0.094 seconds
Preparing documents... 0 line(s) ignored
Running ROUGE...
---------------------------------------------
1 ROUGE-1 Average_R: 0.07895 (95%-conf.int. 0.07895 - 0.07895)
1 ROUGE-1 Average_P: 0.33333 (95%-conf.in

100%|██████████| 152/152 [00:14<00:00, 10.81it/s]


### BART-large ▶ fine-tuned on XSum ▶ fine-tuned on SciTLDR using CATTS framework (abstract only):

In [ ]:
! gsutil cp gs://skiff-models/scitldr/catts-xsum.tldr-ao.pt biotldr/models/catts-xsum.tldr-ao.pt

Copying gs://skiff-models/scitldr/catts-xsum.tldr-ao.pt...
- [1 files][  3.8 GiB/  3.8 GiB]   45.7 MiB/s                                   
Operation completed over 1 objects/3.8 GiB.                                      


In [ ]:
%%bash

cd biotldr/
cp -r SciTLDR-Data/SciTLDR-A/ctrl-bin/ BioTLDR-Data/BioTLDR-A/ctrl-bin/
python scripts/generate.py models/ BioTLDR-Data/BioTLDR-A/ctrl ./outputs/ --checkpoint_file catts-xsum.tldr-ao.pt --beam 4 --lenpen 0.2 --test_fname catts_xsum_ao_domain_transfer.hypo

loading archive file models/
loading archive file BioTLDR-Data/BioTLDR-A/ctrl-bin
| [source] dictionary: 50264 types
| [target] dictionary: 50264 types
Time to run script: 49.01143288612366 sec


152it [00:23,  6.34it/s]


In [63]:
%%bash 

cd biotldr/
python scripts/cal-rouge.py outputs/catts_xsum_ao_domain_transfer.hypo BioTLDR-Data/BioTLDR-A/test.jsonl --workers 1

Preparing documents... 0 line(s) ignored
Running ROUGE...
---------------------------------------------
1 ROUGE-1 Average_R: 0.19048 (95%-conf.int. 0.19048 - 0.19048)
1 ROUGE-1 Average_P: 0.33333 (95%-conf.int. 0.33333 - 0.33333)
1 ROUGE-1 Average_F: 0.24243 (95%-conf.int. 0.24243 - 0.24243)
---------------------------------------------
1 ROUGE-2 Average_R: 0.02439 (95%-conf.int. 0.02439 - 0.02439)
1 ROUGE-2 Average_P: 0.04348 (95%-conf.int. 0.04348 - 0.04348)
1 ROUGE-2 Average_F: 0.03125 (95%-conf.int. 0.03125 - 0.03125)
---------------------------------------------
1 ROUGE-L Average_R: 0.09524 (95%-conf.int. 0.09524 - 0.09524)
1 ROUGE-L Average_P: 0.16667 (95%-conf.int. 0.16667 - 0.16667)
1 ROUGE-L Average_F: 0.12121 (95%-conf.int. 0.12121 - 0.12121)

Elapsed time: 0.097 seconds
Preparing documents... 0 line(s) ignored
Running ROUGE...
---------------------------------------------
1 ROUGE-1 Average_R: 0.13158 (95%-conf.int. 0.13158 - 0.13158)
1 ROUGE-1 Average_P: 0.29412 (95%-conf.in

100%|██████████| 153/153 [00:14<00:00, 10.70it/s]


### BART-large ▶ fine-tuned on XSum ▶ fine-tuned on SciTLDR using CATTS framework (abstract, introduction, conclusion):

In [ ]:
! gsutil cp gs://skiff-models/scitldr/catts-xsum.tldr-aic.pt biotldr/models/catts-xsum.tldr-aic.pt

Copying gs://skiff-models/scitldr/catts-xsum.tldr-aic.pt...
- [1 files][  3.8 GiB/  3.8 GiB]   46.5 MiB/s                                   
Operation completed over 1 objects/3.8 GiB.                                      


In [ ]:
%%bash

cd biotldr/
cp -r SciTLDR-Data/SciTLDR-AIC/ctrl-bin/ BioTLDR-Data/BioTLDR-AIC/ctrl-bin/
python scripts/generate.py models/ BioTLDR-Data/BioTLDR-AIC/ctrl ./outputs/ --checkpoint_file catts-xsum.tldr-aic.pt --beam 2 --lenpen 0.2 --test_fname catts_xsum_aic_domain_transfer.hypo

loading archive file models/
loading archive file BioTLDR-Data/BioTLDR-AIC/ctrl-bin
| [source] dictionary: 50264 types
| [target] dictionary: 50264 types
Time to run script: 46.40085506439209 sec


151it [00:22,  6.77it/s]


In [64]:
%%bash 

cd biotldr/
python scripts/cal-rouge.py outputs/catts_xsum_aic_domain_transfer.hypo BioTLDR-Data/BioTLDR-AIC/test.jsonl --workers 1

Preparing documents... 0 line(s) ignored
Running ROUGE...
---------------------------------------------
1 ROUGE-1 Average_R: 0.21429 (95%-conf.int. 0.21429 - 0.21429)
1 ROUGE-1 Average_P: 0.40909 (95%-conf.int. 0.40909 - 0.40909)
1 ROUGE-1 Average_F: 0.28125 (95%-conf.int. 0.28125 - 0.28125)
---------------------------------------------
1 ROUGE-2 Average_R: 0.02439 (95%-conf.int. 0.02439 - 0.02439)
1 ROUGE-2 Average_P: 0.04762 (95%-conf.int. 0.04762 - 0.04762)
1 ROUGE-2 Average_F: 0.03226 (95%-conf.int. 0.03226 - 0.03226)
---------------------------------------------
1 ROUGE-L Average_R: 0.11905 (95%-conf.int. 0.11905 - 0.11905)
1 ROUGE-L Average_P: 0.22727 (95%-conf.int. 0.22727 - 0.22727)
1 ROUGE-L Average_F: 0.15625 (95%-conf.int. 0.15625 - 0.15625)

Elapsed time: 0.097 seconds
Preparing documents... 0 line(s) ignored
Running ROUGE...
---------------------------------------------
1 ROUGE-1 Average_R: 0.07895 (95%-conf.int. 0.07895 - 0.07895)
1 ROUGE-1 Average_P: 0.60000 (95%-conf.in

100%|██████████| 152/152 [00:13<00:00, 10.94it/s]


## Human Evaluation

In [ ]:
! pip install xlsxwriter
! pip install xlwt

In [1]:
from os.path import join
import pandas as pd

def prepare_human_evaluation_sheet(gen_fname, data_dir, exp, excel_writer):
  with open(gen_fname, 'r') as file:
    generated_TLDRs = [line.strip() for line in file.readlines()]
  with open(join(data_dir, 'test.source')) as file:
    inputs = [line.strip() for line in file.readlines()]
  with open(join(data_dir, 'test.target')) as file:
    gold_TLDRs = [line.strip() for line in file.readlines()]

  human_eval_dict = {
      'predicted': generated_TLDRs,
      'gold': gold_TLDRs,
      'input': inputs
  }

  df = pd.DataFrame.from_dict(human_eval_dict)
  df.to_excel(excel_writer, sheet_name=exp, index=False)

In [ ]:
writer = pd.ExcelWriter('human-evaluation.xlsx', engine='xlsxwriter')

prepare_human_evaluation_sheet('biotldr/outputs/catts_ao_domain_transfer.hypo',
                                'biotldr/BioTLDR-Data/BioTLDR-A/',
                                'catts-ao',
                                writer)
prepare_human_evaluation_sheet('biotldr/outputs/catts_aic_domain_transfer.hypo',
                                'biotldr/BioTLDR-Data/BioTLDR-AIC/',
                                'catts-aic',
                                writer)
prepare_human_evaluation_sheet('biotldr/outputs/catts_xsum_ao_domain_transfer.hypo',
                                'biotldr/BioTLDR-Data/BioTLDR-A/',
                                'catts-xsum-ao',
                                writer)
prepare_human_evaluation_sheet('biotldr/outputs/catts_xsum_aic_domain_transfer.hypo',
                                'biotldr/BioTLDR-Data/BioTLDR-AIC/',
                                'catts-xsum-aic',
                                writer)

writer.save()